In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import altair as alt
import numpy as np
import pandas as pd

alt.data_transformers.enable('default', max_rows=None)
sns.set(color_codes=True)

%matplotlib inline
import sys
sys.path.append("..") # to import from parent directory
from dataset import CSIDataset
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
import logging
from tqdm import tqdm

In [26]:

class AnalyseLSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""

    def __init__(self, input_dim, hidden_dim, output_dim, bidirectional, batch_size):
        super(AnalyseLSTMClassifier, self).__init__()
        self.arch = "lstm"
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_dir = 2 if bidirectional else 1

        self.ILayer = nn.LSTMCell(input_dim, hidden_dim)
        self.layer1 = nn.LSTMCell(hidden_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim*self.num_dir, hidden_dim)
        self.layer3 = nn.ReLU(True)
        self.layer4 = nn.Linear(hidden_dim, hidden_dim)
        self.layer5 = nn.ReLU(True)
        self.OLayer = nn.Linear(hidden_dim, output_dim)
        self.hidden = None

    def forward(self, x):
        seq_len, batch_size, _ = x.size()
        if self.hidden is None:
            self.hidden = self.init_hidden(batch_size)
            h0, c0, h1, c1 = self.hidden
        else:
            h0, c0, h1, c1 = self.hidden
        outputIL, outputL1 = [], []

        for t in range(seq_len):
            h0, c0 = self.ILayer(x[t], (h0, c0))
            outputIL.append(h0)
            h1, c1 = self.layer1(h0, (h1, c1))
            outputL1.append(h1)
        outputIL = torch.stack(outputIL, dim=0)
        outputL1 = torch.stack(outputL1, dim=0)
        outputL2 = self.layer2(outputL1[-1,:,:])
        outputL3 = self.layer3(outputL2)
        outputL4 = self.layer4(outputL3)
        outputL5 = self.layer5(outputL4)
        outputL = self.OLayer(outputL5) #class probabilities

        return outputL

    def init_hidden(self, batch_size):
        h0 = nn.Parameter(nn.init.xavier_uniform_(
                torch.Tensor(batch_size, self.hidden_dim).type(torch.DoubleTensor)
            ), requires_grad=True).to(device)

        c0 = nn.Parameter(nn.init.xavier_uniform_(
                torch.Tensor(batch_size, self.hidden_dim).type(torch.DoubleTensor)
            ), requires_grad=True).to(device)
        h1 = nn.Parameter(nn.init.xavier_uniform_(
                torch.Tensor(batch_size, self.hidden_dim).type(torch.DoubleTensor)
            ), requires_grad=True).to(device)

        c1 = nn.Parameter(nn.init.xavier_uniform_(
                torch.Tensor(batch_size, self.hidden_dim).type(torch.DoubleTensor)
            ), requires_grad=True).to(device)

        return h0, c0, h1, c1

In [27]:
def get_train_metric_demo(model, dl, criterion, BATCH_SIZE):
    model.eval()

    correct, total, total_loss = 0, 0, 0

    model.hidden = model.init_hidden(BATCH_SIZE)
    for x_val, y_val in tqdm(dl, total=len(dl), desc="Validation epoch: "):
        if x_val.size(0) != BATCH_SIZE:
            continue
        model.init_hidden(x_val.size(0))
        x_val = x_val.permute(1, 0, 2)
        
        x_val = x_val.double().to(device)
        y_val = y_val.double().to(device)

        out = model(x_val)

        loss = criterion(out, y_val.long())

        total_loss += loss.item()

        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()

    acc = correct / total

    return total_loss, correct, total, acc

In [28]:

logging.basicConfig(level=logging.INFO)

# Cuda support
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

logging.info("Device: {}".format(device))

INFO:root:Device: cpu


In [29]:
input_dim = 468
hidden_dim = 256
dropout = 0.0
output_dim = 7
batch_size = 4
model = AnalyseLSTMClassifier(input_dim, hidden_dim, output_dim, False, batch_size).double().to(device)
EPOCHS_NUM = 2
BATCH_SIZE = 4
SEQ_DIM = 1024
DATA_STEP = 1

class_weights = (
    torch.Tensor([0.113, 0.439, 0.0379, 0.1515, 0.0379, 0.1212, 0.1363])
    .double()
    .to(device)
)
class_weights_inv = 1 / class_weights

criterion = nn.CrossEntropyLoss(weight=class_weights_inv)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00146)
scheduler = ReduceLROnPlateau(optimizer, "min", factor=0.5)
patience, trials, best_acc = 100, 0, 0

In [30]:
def load_data():
    logging.info("Loading data...")

    train_dataset = CSIDataset(
        [
            "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_1\\1",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_1\\2",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_1\\3",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_1\\4",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_2\\1",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_2\\2",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_3\\1",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_3\\2",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_3\\3",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_3\\4",
            # "D:\\Gitdesktop\\KLTN\\pythonFile\\data\\room_3\\5",
        ],
        SEQ_DIM,
        DATA_STEP,
    )

    val_dataset = train_dataset

    logging.info("Data is loaded...")

    trn_dl = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0
    )
    val_dl = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0
    )

    return trn_dl, val_dl

In [ ]:
trn_dl, val_dl = load_data()
logging.info("Start model training")
for epoch in range(1, EPOCHS_NUM + 1):
    model.train(mode=True)
    for i, (x_batch, y_batch) in tqdm(enumerate(trn_dl), total=len(trn_dl), desc="Training epoch: "):
        if x_batch.size(0) != 4:
            continue
        x_batch = x_batch.permute(1, 0, 2).double().to(device) # (seq_len, batch, input_size) 
        print(x_batch.shape)
        print("x_batch at seq 0: ",x_batch[0]) #input data
        print("x_batch at seq 1: ",x_batch[1])
        output = model(x_batch)
        print("Output: ",output) #output probabilities of next 4 packets 
        print("Predict: ",torch.argmax(torch.nn.functional.log_softmax(output, dim=1), dim=1))  # predicted class
        print("y_batch: ",y_batch) #actual class  
        loss = criterion(output, y_batch.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i == 1:
            break
    break
    val_loss, val_correct, val_total, val_acc = get_train_metric_demo(
            model, val_dl, criterion, batch_size
    )
    train_loss, train_correct, train_total, train_acc = get_train_metric_demo(
        model, trn_dl, criterion, batch_size
    )

    logging.info(
    f"Epoch: {epoch:3d} |"
    f" Validation Loss: {val_loss:.2f}, Validation Acc.: {val_acc:2.2%}, "
    f"Train Loss: {train_loss:.2f}, Train Acc.: {train_acc:2.2%}"
    )
    if val_acc > best_acc:
        trials = 0
        best_acc = val_acc
        torch.save(model.state_dict(), "D:\\Gitdesktop\\WIFI_CSI_based_HAR\\model\\saveModels\\lstm_best.pth")
        logging.info(
            f"Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}"
        )
    else:
        trials += 1
        if trials >= patience:
            logging.info(f"Early stopping on epoch {epoch}")
            break
    scheduler.step(val_loss)

INFO:root:Loading data...
INFO:root:Data is loaded...
INFO:root:Start model training
Training epoch:   0%|          | 0/1052 [00:00<?, ?it/s]

torch.Size([1024, 4, 468])
Output:  tensor([[ 0.0087, -0.0068, -0.0407, -0.0395, -0.0315,  0.0582,  0.0368],
        [ 0.0077, -0.0059, -0.0436, -0.0413, -0.0310,  0.0573,  0.0349],
        [ 0.0081, -0.0062, -0.0425, -0.0420, -0.0321,  0.0580,  0.0353],
        [ 0.0082, -0.0064, -0.0417, -0.0419, -0.0302,  0.0577,  0.0354]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)
Predict:  tensor([5, 5, 5, 5])
y_batch:  tensor([5, 1, 1, 0])


Training epoch:   0%|          | 0/1052 [00:04<?, ?it/s]
